# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use **Pandas** functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

# Imports

In [1]:
# Standard libs
import pandas as pd
import numpy as np

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located. When reading the file use the types specified in the course assignment for columns of the data.

In [2]:
sales_dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int,
               'sqft_living15':float, 'grade':int, 'yr_renovated':int,
               'price':float, 'bedrooms':float, 'zipcode':str, 'long':float,
               'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int,
               'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str,
               'sqft_lot':int, 'view':int}

sales = pd.read_csv("./data/kc_house_data.csv", dtype=sales_dtype)

In [3]:
sales.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
5,7237550310,20140512T000000,1225000.0,4.0,4.50,5420.0,101930,1,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760.0,101930.0
6,1321400060,20140627T000000,257500.0,3.0,2.25,1715.0,6819,2,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238.0,6819.0
7,2008000270,20150115T000000,291850.0,3.0,1.50,1060.0,9711,1,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650.0,9711.0
8,2414600126,20150415T000000,229500.0,3.0,1.00,1780.0,7470,1,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780.0,8113.0
9,3793500160,20150312T000000,323000.0,3.0,2.50,1890.0,6560,2,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390.0,7570.0


# Load training and test data

As not using GraphLab, we load training and test data from the files provided with the course assignment. 

In [4]:
train_data = pd.read_csv("./data/kc_house_train_data.csv", dtype=sales_dtype)
test_data = pd.read_csv("./data/kc_house_test_data.csv", dtype=sales_dtype)

# Build a generic simple linear regression function 

The function computes the **intercept** and **slope** of the simple linear regression from the closed form solution (gradient = 0). What needs to be compute is:
* $\sum_{i=0}^N X_i$
* $\sum_{i=0}^N Y_i$
* $\sum_{i=0}^N Y_iX_i$
* $\sum_{i=0}^N X_i^2$

In [5]:
def simple_linear_regression(input_feature, output):
    
    n = input_feature.shape[0]
    sum_xi = input_feature.sum()
    sum_yi = output.sum()
    sum_yixi = (input_feature*output).sum()
    sum_xi2 = (input_feature*input_feature).sum()
    
    slope = (sum_yixi - sum_xi*sum_yi/n)/(sum_xi2 - sum_xi*sum_xi/n)
    intercept = sum_yi/n - slope*sum_xi/n
    
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [6]:
test_feature = np.arange(5)
test_output = 1 + 1*test_feature
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print("Intercept:",str(test_intercept))
print("Slope:",str(test_slope))

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [7]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data["sqft_living"], train_data["price"])

print("Intercept:",str(sqft_intercept))
print("Slope:",str(sqft_slope))

Intercept: -47116.07907289418
Slope: 281.9588396303426


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using Pandas it's easy to multiply an array by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [8]:
def get_regression_predictions(input_feature, intercept, slope):
    
    predicted_values = intercept + slope*input_feature
    
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [9]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print("The estimated price for a house with",my_house_sqft,"squarefeet is:",round(estimated_price,2))

The estimated price for a house with 2650 squarefeet is: 700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [10]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    
    y_predict = get_regression_predictions(input_feature, intercept, slope)
    RSS = np.square(y_predict - output).sum()

    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [11]:
print("RSS: ",get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope))

RSS:  0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model, what is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [12]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print("The RSS of predicting Prices based on Square Feet is :",rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 1201918354177286.2


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Comlplete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output!

In [13]:
def inverse_regression_predictions(output, intercept, slope):
    
    estimated_feature = (output - intercept)/slope

    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [14]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print("The estimated squarefeet for a house worth",my_house_price,"is:",round(estimated_squarefeet,2))

The estimated squarefeet for a house worth 800000 is: 3004.4


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [15]:
bedrooms_intercept, bedrooms_slope = simple_linear_regression(train_data["bedrooms"], train_data["price"])

print("Intercept:",str(bedrooms_intercept))
print("Slope:",str(bedrooms_slope))

Intercept: 109473.1776229596
Slope: 127588.95293398784


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [16]:
# Compute RSS when using bedrooms on TEST data:
rss_test_prices_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'],
                                                          test_data['price'], bedrooms_intercept, bedrooms_slope)
print("The RSS [test data] of predicting Prices based on Square Feet is :",rss_test_prices_on_bedrooms)

The RSS [test data] of predicting Prices based on Square Feet is : 493364585960301.4


In [17]:
# Compute RSS when using squarefeet on TEST data:
rss_test_prices_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print("The RSS [test data] of predicting Prices based on Square Feet is :",rss_test_prices_on_sqft)

The RSS [test data] of predicting Prices based on Square Feet is : 275402933617813.1
